##### **Brief introduction**

This post was really just an addition towards the last random forest (RF) post (machine learning (ML) series 2.2). It was mainly inspired by this paper (cite Esposito et al. on ghostml) from [rinikerlab](https://github.com/rinikerlab). It was also nice to complete the RF series by adding a RF classifier model since last post was only on a regressor. Knowing that imbalanced datasets were common in drug discovery projects, learning new strategies to deal with them was also extremely useful, and while I was working on this post, I also came across a few other packages that I haven't used before, so I've included them all down below.

**Mention outcome here? - change in Cohen's kappa & AUC or improvement in confusion matrix after using ghostml, along with change in ROC curve**

<br>

##### **Overview on packages/scripts used**

* Data source using *chembl_downloader*
* Own little script of random_forest.py (avoid repeating code)
* SMILES checker from *scikit_mol*
* Dealing with imbalanced datasets in RF classifiers by using *ghostml*

<br>

##### **Importing libraries**


In [ ]:
import pandas as pd
import numpy as np
import chembl_downloader
from chembl_downloader import latest, queries, query
import datamol as dm
from rdkit.Chem import Descriptors
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scikit_mol.utilities import CheckSmilesSanitazion
import ghostml

<br>

##### **Data retrieval using chembl_downloader**

chembl_downloader package was something I wanted to try a while back after knowing that it would thoroughly document the ChEMBL data source (I've tried manual download and chembl_webresource_client, and they were probably not the best strategies for data reproducibility). Its idea was to generate a data source that could be fully reproducible. It involved some SQL at the beginning to specify the exact type of data needed, so some SQL knowledge was required. The rest were pretty straightforward. Other uses for this package were elaborated much more nicely in its GitHub repository at https://github.com/cthoyt/chembl-downloader. Overall, I think it was very useful in a Jupyter notebook or equivalent environments to document the data used.

Reference notebooks that used chembl_downloader (more provided in its repository):
- https://github.com/cthoyt/chembl-downloader/blob/main/notebooks/drug-indications.ipynb
- https://github.com/PatWalters/practical_cheminformatics_tutorials/blob/1f7c61f83eec81081ef2605ac70440bf1940d914/misc/working_with_ChEMBL_drug_data.ipynb#L80

What I did was shown below.


In [ ]:
# Show the latest version of ChEMBL used
latest_version = latest()
print(f"The latest ChEMBL version is: {latest_version}")

In [ ]:
# Query chembl_downloader to show SQL required to extract ChEMBL data for a specific protein target
# e.g. target_chembl_id for AChE: CHEMBL220
queries.markdown(queries.get_target_sql(target_id="CHEMBL220", target_type="SINGLE PROTEIN"))

```{{python}}
# Following data download worked (4.41GB for SQL query below, might take several min)

# Added MOLECULE_DICTIONARY.max_phase into sql to show max phases for compounds targeting AChE

sql = """
SELECT
    ASSAYS.chembl_id              AS assay_chembl_id,
    TARGET_DICTIONARY.target_type,
    TARGET_DICTIONARY.tax_id,
    TARGET_DICTIONARY.chembl_id,
    COMPOUND_STRUCTURES.canonical_smiles,
    MOLECULE_DICTIONARY.chembl_id AS molecule_chembl_id,
    MOLECULE_DICTIONARY.max_phase,
    ACTIVITIES.standard_type,
    ACTIVITIES.pchembl_value
FROM TARGET_DICTIONARY
     JOIN ASSAYS ON TARGET_DICTIONARY.tid == ASSAYS.tid
     JOIN ACTIVITIES ON ASSAYS.assay_id == ACTIVITIES.assay_id
     JOIN MOLECULE_DICTIONARY ON MOLECULE_DICTIONARY.molregno == ACTIVITIES.molregno
     JOIN COMPOUND_STRUCTURES ON MOLECULE_DICTIONARY.molregno == COMPOUND_STRUCTURES.molregno
WHERE TARGET_DICTIONARY.chembl_id = 'CHEMBL220'
    AND ACTIVITIES.pchembl_value IS NOT NULL
    AND TARGET_DICTIONARY.target_type = 'SINGLE PROTEIN'
"""

df = chembl_downloader.query(sql)
```

```{{python}}
df.head()
```


In [ ]:
# Save df as .csv file
#df.to_csv("chembl_d_ache", sep=",", index=False)

In [ ]:
# Load dataset from saved .csv file
df_ache = pd.read_csv("chembl_d_ache")
print(df_ache.shape)
df_ache.head()

<br>

##### **Some data cleaning**

Minor cleaning and preprocessing were done for this post only, as the focus was more on dealing with imbalanced datasets in RF classifier.

<br>

###### **mol_prep.py**

I've written or more like compiled my own little pieces of code into a Python script. The idea was to remove most function code in the post to avoid repeating them as they've been used frequently in the last few posts. The script would be saved into my project repository, and it would still be a "work-in-progress" script as I work further in the future.


In [ ]:
## Trial own mol_prep.py script
from mol_prep import preprocess, rdkit_2d_descriptors

In [ ]:
## Preprocess/standardise molecules
# Running preprocess function 
df_ache = df_ache.copy()
df_prep = df_ache.apply(preprocess, axis = 1)
df_prep.head()

<br>

###### **scikit_mol**

Scikit_mol was a package originated from 2022 RDKit UGM hackathon. This [blog post](https://www.cheminformania.com/scikit-mol-easy-embedding-of-rdkit-into-scikit-learn/) elaborated further on its functions and uses in machine learning. For this post I've only used it for a very small portion, mainly to check for missing SMILES or errors in SMILES (kind of like double checking whether my preprocess function code worked as expected). It could be integrated with scikit-learn's pipeline method on multiple estimators. GitHub Repository link: https://github.com/EBjerrum/scikit-mol


In [ ]:
# Quick simple way to check for missing SMILES
print(f'Dataset contains {df_prep.standard_smiles.isna().sum()} unparsable mols')

It appeared to have no unparsable (missing) molecules.


In [ ]:
# Checking for invalid SMILES using scikit_mol
smileschecker = CheckSmilesSanitazion()

smileschecker.sanitize(list(df_prep.standard_smiles))

# Showing SMILES errors
smileschecker.errors

It also confirmed that there were no errors in SMILES (as a second confirmation that the preprocessed molecules had no other unusual errors after using Datamol).


In [ ]:
## Generate RDKit2D descriptors/fingerprints
# Running rdkit_2d_descriptors function
df_2d = rdkit_2d_descriptors(df_prep)
df_2d

In [ ]:
# Merge dataframes df_prep & df_2d via index
df_merge = pd.merge(
    df_prep[["max_phase", "molecule_chembl_id"]],
    df_2d,
    left_index=True,
    right_index=True
)

In [ ]:
print(df_merge.shape)
df_merge.head()

Different spreads of max phases were shown as the SQL query mainly used IC50 (whereas last post was strictly limited to Ki via ChEMBL web resource client). Other reason was that in the decision tree series, I attempted data preprocessing at a larger scale (which eliminated some data). So, it appeared that there were more max phase 4 compounds here than last time (Note: null compounds were not shown in the value counts here as it was labelled as "NaN", but it should be the largest portion of max phase in the data).


In [ ]:
# Find out counts of each max phase
df_merge.value_counts("max_phase")

I then tried searching for the chembl_id of the 10 max phase 4 compounds used in the last post in df_merge.


In [ ]:
# Previously used 10 max phase 4 compounds
# donepezil = CHEMBL502 
# galantamine = CHEMBL659
list_mp4 = ["CHEMBL95", "CHEMBL1128", "CHEMBL640", "CHEMBL502", "CHEMBL481", "CHEMBL360055", "CHEMBL1025", "CHEMBL659", "CHEMBL1200970", "CHEMBL1677"]

# Search for compounds in list_mp4 within df_merge's "molecule_chembl_id" column
# using Series.isin
df_prev = df_merge.loc[df_merge["molecule_chembl_id"].isin(list_mp4)]
df_prev

There were many duplicates of compounds.


In [ ]:
# Some compounds with duplicates
print(df_prev.shape)
df_prev.value_counts("molecule_chembl_id")

In [ ]:
# Dropping duplicated compound via chembl IDs in the main df
df_merge_new = df_merge.drop_duplicates(subset=["molecule_chembl_id"], keep="first")
print(df_merge_new.shape)
df_merge_new.head()

In [ ]:
# Previously used 10 max phase 4 compounds were found in df_merge_new
df_mp4 = df_merge_new.loc[df_merge_new["molecule_chembl_id"].isin(list_mp4)]
df_mp4

In [ ]:
# note: compounds with max phase 0 not shown in the count
df_merge_new.value_counts("max_phase")

<br>

##### **Model building**

The aim was to model and classify the max phase of ChEMBL small molecules - max phase 4 or not:
    - *target*: max_phase
    - *features*: various RDKit 2D descriptors (RDKit2D)

1. Re-labelled max phases as binary labels (e.g. max phase null as 0, max phase 4 as 1) 


In [ ]:
# Re-label max phase NaN as 0
df_merge_new = df_merge_new.fillna(0)
df_merge_new

2. Splitting data into max phase null & max phase 4 (reason being needing to re-label max phase 4 column only as 1, and not disrupting max phase 0 compounds)


In [ ]:
# Select all max phase null compounds
df_null = df_merge_new[df_merge_new["max_phase"] == 0]
print(df_null.shape)
df_null.head()

In [ ]:
# Using pd.DataFrame.assign to add a new column to re-label max_phase 4 into "1"
df_mp4_lb = df_mp4.assign(max_phase_lb = df_mp4["max_phase"] / 4)

# Using pd.DataFrame.pop() & insert() to shift added column to first column position
first_col = df_mp4_lb.pop("max_phase_lb")
df_mp4_lb.insert(0, "max_phase_lb", first_col)

df_mp4_lb

In [ ]:
# Also create a new column max_phase_lb column for df_null to merge 2 dfs together

df_null_lb = df_null.assign(max_phase_lb = df_null["max_phase"])
first_col_null = df_null_lb.pop("max_phase_lb")
df_null_lb.insert(0, "max_phase_lb", first_col_null)
df_null_lb.head()

The two dataframes should share same column names which could be combined together.


In [ ]:
# Concatenate df_mp4_lb & df_null_lb
df_full = pd.concat([df_null_lb, df_mp4_lb])
df_full

3. Define X and y variables and train RF classifier model 

Eventually df_full contained 10 active compounds and 5256 inactive compounds, as shown from the value counts.


In [ ]:
df_full.value_counts("max_phase_lb")

In [ ]:
# Cast max_phase_lb column into integer
df_full["max_phase_lb"] = df_full["max_phase_lb"].astype(int)

In [ ]:
df_full.head()

In [ ]:
# Defining X (features) & y (target)
X = df_full.iloc[:, 3:]
y = df_full.iloc[:, 0]

In [ ]:
# Checking the right data have been selected
y

In [ ]:
# Convert both X & y to arrays
X = X.to_numpy()
y = y.to_numpy()

In [ ]:
# Using train_test_split() this time to split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

* Training RF classifier model

Reference notebook: https://github.com/rinikerlab/GHOST/blob/main/notebooks/example_GHOST.ipynb


In [ ]:
# Using RandomForestClassifier() to train model
# max_features = "sqrt" by default, I decided to show it explicitly in the code to make sure the right one was used
rfc = RandomForestClassifier(max_depth=3, random_state=1, max_features="sqrt", oob_score=True)
rfc.fit(X_train, y_train)

4. Obtain the prediction probabilities on the testing data and show confusion matrix with classification metrics


In [ ]:
# Extract positive prediction probabilities for the testing set
test_probs = rfc.predict_proba(X_test)[:, 1]

A useful web link on area under the receiver operating characteristic curve - https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc


In [ ]:
# Rather than re-inventing the wheel, the following function code for calculating metrics of the RF model was adapted from Landrum et al., from this notebook: https://github.com/rinikerlab/GHOST/blob/main/notebooks/example_GHOST.ipynb

def calc_metrics(y_test, test_probs, threshold = 0.5):
    # Scores to differentiate between stated decision threshold (default = 0.5)
    scores = [1 if x>=threshold else 0 for x in test_probs]
    # Calculate area under the receiver operating characteristic curve
    auc = metrics.roc_auc_score(y_test, test_probs)
    # Calculate Cohen's Kappa score
    kappa = metrics.cohen_kappa_score(y_test, scores)
    # Formulate the confusion matrix
    confusion = metrics.confusion_matrix(y_test, scores, labels = list(set(y_test)))
    print('thresh: %.2f, kappa: %.3f, AUC test-set: %.3f'%(threshold, kappa, auc))
    print(confusion)
    print(metrics.classification_report(y_test,scores))
    return 

Initially I made a mistake when doing the data split, which created a problem with y_test which only contained one label in it (1 only), needs binary labels (0 & 1) 

It was needed for roc_auc_score (measures true +ve & false +ve rates).


In [ ]:
# Show confusion matrix along with AUC and Cohen's Kappa
calc_metrics(y_test, test_probs, threshold = 0.5)

# This was an extreme case - as only 10 actives vs. 5256 inactives

5. two approaches from GHOST paper (main idea was optimising and shifting decision threshold):

    - approach 1 (out-of-bag method, aimed for RF classifiers) based on RDKit blog post (ref. 41) - http://rdkit.blogspot.com/2018/11/working-with-unbalanced-data-part-i.html ([viewable notebook version](https://nbviewer.org/github/greglandrum/rdkit_blog/blob/master/notebooks/Working%20with%20unbalanced%20data%20part%201.ipynb) via nbviewer)

    - approach 2 led to Generalised threshold shifting (GHOST) procedure (could be used for any classification methods) - ghostml package

I only used approach 2 here as both approaches were shown to be performing similarly in the paper, and also approach 1 was already described in a RDKit blog post.

* extract the prediction probabilities from the RF classifier trained model


In [ ]:
# Get the positive prediction probabilities of the training set
train_probs = rfc.predict_proba(X_train)[:, 1]

6. Shift decision threshold using ghostml in a postprocessing way (note: last post used re-sampling method in a preprocessing way)

* optimise the decision threshold using ghostml via testing various thresholds (in spaces of 0.05 with range of 0.05 to 0.5) - to search for the most optimal threshold with most maximised Cohen's kappa


In [ ]:
# Setting up different decision thresholds in space of 0.05
thresholds = np.round(np.arange(0.05,0.55,0.05),2)
thresholds

In [ ]:
# Looking for the best threshold with the most optimal Cohen's Kappa
new_threshold = ghostml.optimize_threshold_from_predictions(y_train, train_probs, thresholds, ThOpt_metrics = 'ROC') 

In [ ]:
# Show the optimal decision threshold
new_threshold

* calculate confusion matrix and classification metrics based on the optimised decision threshold


In [ ]:
# Using calc_metrics function again on the newly-found/shifted decision threshold
# It showed an improved classification outcome through the confusion matrix
calc_metrics(y_train, train_probs, threshold=new_threshold)

<br>

##### **ROC curves**


In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import RocCurveDisplay, roc_curve

###### **Testing set ROC curve**


In [ ]:
# Testing set ROC curve
RocCurveDisplay.from_predictions(y_test, test_probs, plot_chance_level = True)

<br>

###### **Training set ROC curve**


In [ ]:
# Training set ROC curve
RocCurveDisplay.from_predictions(y_train, train_probs, plot_chance_level = True)

Alternative ROC curve plotting method that would reproduce a similar plot:


In [ ]:
# Look for true positive rate (tpr), false positive rate (fpr) & threshold
fpr, tpr, thresh = metrics.roc_curve(y_train, train_probs)
# Plotting
plt.figure()
# lw = linewidth
plt.plot(fpr, tpr, lw = 2)
# show random guessing line (threshold = 0.5)
plt.plot([0, 1], [0, 1], color = "g", lw = 2, linestyle="--")
plt.ylim([-0.05, 1.05])
plt.xlim([-0.05, 1.0])
plt.xlabel("specificity")
plt.ylabel("recall")
plt.show()